Si vous n'avez pas Selenium sur votre ordinateur, il est recommandé de l'installer avec la cellule en dessous.

In [ ]:
#!pip install -r requirements.txt

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from ast import literal_eval  # Importer cette fonction pour évaluer les chaînes de caractères comme des dictionnaires
import math 

# Préambule

Notre travail sur le traitement des données se divise en 3 grandes parties:

### 1- Dataset Info_rank

Le but premier de ce dataset est de regrouper dans un même fichier tous les joueurs ainsi que leur rang. Pour se faire, on va récupérer pour chaque année le top 100 des joueurs, qui est jusque là mis sous forme d'identifiant et  le relier aux informations du joueurs grâce à son ID.

pour créer ce dataset nous allons:
- $\textbf{concaténer}$ les fichiers atp_rankings 
- $\textbf{merge}$ le fichier obtenu avec les informations contenues dans atp_players
- $\textbf{nettoyer}$ le dataset en filtrant les données utiles 

### 2- Dataset info_match

À travers ce dataset, nous regroupons tous les matchs des joueurs du top 100 ainsi que diverses informations telles que:

- Vainqueur / Perdant
- Rang du vainqueur / Rang du perdant
- surface du match
- tournois

### 3- Dataset info_joueur

Ce dataset est vraiment le centre de notre projet. Il regroupe toutes les informations des joueurs du top 100 par année. Que ce soit leur taille, rang, nombre de victoires etc... Celui-ci va nous permettre d'analyser ce qui différencie un joueur du top 10 d'un joueur du top 50.

La plupart des informations obtenues ont été calculées grâce au dataset info_match et aux données que nous avons $\textbf{scrapé}$ 

Le format du dataset obtenu est un dataset qui pour chaque année contient un dictionnaire référençant toutes les statistiques du joueur sur cette année. Ainsi une ligne correspond à l'ensemble des informations que nous possédons sur la carrière d'un joueur (d'un point de vu sportif).

# Informations pratiques

- L'ensemble des données qui résultent d'un traitement de notre part et qui nous seront utiles pour la suite du projet sont stockées dans $\textbf{/Data/Data_utiles}$ . Nous avons fait le choix de conserver les données "brutes" pour mettre en lumière le travail effectuées pour la création des datasets finaux.

- les codes de scraping sont accessibles dans $\textbf{/Data/scrapping}$. Nous avons utilisé le module selenium pour scraper.

# Création du Dataset Info_rank



In [2]:
""" Le but de ce fichier est de récupérer pour chaque année le top 100 des joueurs,
qui est jusque là mis sous forme d'identifiant et de le relier aux informations du joueurs grâce à son ID"""

Years=['00','10','20','90']


liste_concat=[]

for year in Years:

    #On importe le classement des joueurs par année
    df_ranking= pd.read_csv('./Data/Data_brutes/atp_rankings_{}s.csv'.format(year))

    #On supprime les joueurs qui ont un rang superieur à 100 
    df_ranking = df_ranking[df_ranking['rank'] <= 100]



    #On commence par renommer la colonne du nom du joueur histoire de pouvoir merge via une colonne en commun
    df_ranking=df_ranking.rename(columns={'player':'player_id'})

    liste_concat.append(df_ranking.copy())


df_ranking=pd.concat(liste_concat)

#On importe les données relatives à chaque joueur
df_info_joueurs = pd.read_csv('./Data/Data_brutes/atp_players.csv')

#On peut ensuite joindre les deux datasets

df_info_joueurs=pd.merge(df_ranking,df_info_joueurs,on='player_id')
df_info_joueurs.sort_values(['ranking_date','rank'],ascending=True,inplace=True)

df_info_joueurs.to_csv('./Data/Data_utiles/info_rank')

In [4]:
df_infos_rang_joueurs=pd.read_csv('./Data/Data_utiles/info_rank')

df_infos_rang_joueurs.head()


,Unnamed: 0,ranking_date,rank,player_id,points,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,540339,19900101,1,100656,2913.0,Ivan,Lendl,R,19600307.0,USA,188.0,Q182736
1,83014,19900101,2,101414,2279.0,Boris,Becker,R,19671122.0,GER,190.0,Q76334
2,540632,19900101,3,101222,2111.0,Stefan,Edberg,R,19660119.0,SWE,188.0,Q189542
3,541039,19900101,4,100763,1398.0,Brad,Gilbert,R,19610809.0,USA,185.0,Q715134
4,541326,19900101,5,100581,1354.0,John,McEnroe,L,19590216.0,USA,180.0,Q16474


In [6]:
df_infos_rang_joueurs.shape

(162982, 12)

On voit que ce dataset est composé de $12$ colonnes et $162982$ lignes, ce qui est assez conséquent. 

Le but de notre projet étant de définir des caractéristiques qui distinguent un joueur du top 10 d'un autre du top 50, nous pouvons d'ors et déjà supprimer certaines colonnes qui ne nous seront pas utiles telles que:

- IOC vu que le pays d'origine du joueur n'a aucun impact sur ses performances
- wikidata_id
- dob
- Unnamed :0 (qui est un duplicat des indexes)

In [5]:
df_infos_rang_joueurs.drop(['Unnamed: 0', 'ioc', 'wikidata_id','dob'],axis=1,inplace=True)


In [6]:
#On va également fusionner les colonnes name_first et name_last
df_infos_rang_joueurs['Name']=df_infos_rang_joueurs['name_first']+ ' ' +df_infos_rang_joueurs['name_last']

df_infos_rang_joueurs=df_infos_rang_joueurs.drop(['name_first','name_last'],axis=1)

In [7]:
df_infos_rang_joueurs.head()

,ranking_date,rank,player_id,points,hand,height,Name
0,19900101,1,100656,2913.0,R,188.0,Ivan Lendl
1,19900101,2,101414,2279.0,R,190.0,Boris Becker
2,19900101,3,101222,2111.0,R,188.0,Stefan Edberg
3,19900101,4,100763,1398.0,R,185.0,Brad Gilbert
4,19900101,5,100581,1354.0,L,180.0,John McEnroe


In [8]:
df_infos_rang_joueurs.to_csv('./Data/Data_utiles/info_rank.csv')

# Création du Dataset Info_match




In [2]:
liste_concat=[]

for year in range(1993,2022):
    df_matches_annee=pd.read_csv('./Data/Data_brutes/atp_matches_{}.csv'.format(year))

    #On garde seulement les matches des joueurs du top 100

    df_matches=df_matches_annee[(df_matches_annee['winner_rank']<=100) | (df_matches_annee['loser_rank']<=100)].copy()
    
    liste_concat.append(df_matches)

info_matches=pd.concat(liste_concat)
colonnes_a_supprimer = ['winner_ioc', 'winner_entry', 'winner_seed', 'draw_size','loser_ioc', 'loser_entry', 'loser_seed']
info_matches=info_matches.drop(colonnes_a_supprimer,axis=1)

In [10]:
print(info_matches.shape)
print(info_matches.columns)

(80978, 42)
Index(['tourney_id', 'tourney_name', 'surface', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_age', 'loser_id', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_age', 'score', 'best_of', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')


In [3]:
info_matches.head()

,tourney_id,tourney_name,surface,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1993-339,Adelaide,Hard,A,19930104,1,101990,Richard Krajicek,R,196.0,...,32.0,17.0,10.0,8.0,4.0,8.0,10.0,NaN,63.0,NaN
2,1993-339,Adelaide,Hard,A,19930104,3,101890,Jonathan Stark,R,188.0,...,26.0,15.0,9.0,8.0,1.0,5.0,85.0,NaN,54.0,NaN
3,1993-339,Adelaide,Hard,A,19930104,4,102146,Jordi Burillo,R,188.0,...,19.0,12.0,9.0,8.0,1.0,5.0,157.0,NaN,44.0,NaN
4,1993-339,Adelaide,Hard,A,19930104,5,101334,Alexander Volkov,L,188.0,...,43.0,31.0,9.0,10.0,4.0,5.0,17.0,NaN,81.0,NaN
5,1993-339,Adelaide,Hard,A,19930104,6,101685,Markus Naewie,R,190.0,...,35.0,20.0,12.0,9.0,6.0,10.0,77.0,NaN,617.0,NaN


In [11]:
info_matches.to_csv('./Data/Data_utiles/info_matches.csv')

### Fonction pour calculer le winrate d'un joueur sur une année

In [11]:
def winrate(year,_id,dataset):
    
    df_year = dataset[(dataset['tourney_date'] >= year * 10000) & (dataset['tourney_date'] < (year + 1) * 10000)].copy()

    nombre_matches_gagnes=0
    nombre_matches_joues=0
    for index, row in df_year.iterrows():
        
        if row['winner_id']==_id :
            nombre_matches_gagnes+=1
            nombre_matches_joues+=1
            
        elif row['loser_id']==_id:
            nombre_matches_joues+=1
    
    winrate=(nombre_matches_gagnes/nombre_matches_joues)
    
    return winrate,nombre_matches_gagnes,nombre_matches_joues


In [13]:
print(winrate(1993,101948,info_matches))

(0.8415841584158416, 85, 101)


# Création du dataset Info_joueurs

In [3]:
info_matches1=pd.read_csv('./Data/Data_utiles/info_matches.csv')
columns_needed = ['l_1stIn', 'l_svpt', 'l_1stWon', 'l_2ndWon', 'w_svpt', 'w_1stWon', 'w_2ndWon','tourney_date', 'loser_rank', 'winner_rank','loser_rank','w_bpFaced','l_bpFaced','l_bpSaved','w_bpSaved','l_SvGms','w_SvGms']

# Filtrer le DataFrame pour exclure les lignes avec des NaN dans les colonnes spécifiées
info_matches=info_matches1.dropna(subset=columns_needed)

Cette partie du code analyse des données de matches de tennis entre 1993 et 2022. On crée un dictionnaire (joueurs_dict) pour stocker les statistiques des joueurs, puis parcourt chaque match pour mettre à jour ces statistiques. Les informations incluent les pourcentages de services réussis, de jeux gagnés, etc. Les résultats sont organisés par joueur et par année.



In [26]:
# Initialiser le dictionnaire avec des valeurs vides
joueurs_dict = {}


#On commence par créer un dictionnaire dans lequel on met tous les id et noms des joueurs du top 100 depuis 1993
#Pour se faire, on parcourt info_matches ligne par ligne et des qu'on tombe sur un joueur avec un rang inférieur à 
# 100 on l'ajoute au dictionnaire


for index, row in info_matches.iterrows():
    winner_id = row['winner_id']
    loser_id = row['loser_id']

    if (row['winner_rank'] <= 100) and (winner_id not in joueurs_dict):
        joueurs_dict[winner_id] = {'name': row['winner_name'], 'hand': row['winner_hand'], 'height': row['winner_ht']}
        for year in range(1993, 2023):  
            joueurs_dict[winner_id].setdefault(str(year), {})

    if (row['loser_rank'] <= 100) and (loser_id not in joueurs_dict):
        joueurs_dict[loser_id] = {'name': row['loser_name'], 'hand': row['loser_hand'], 'height': row['loser_ht']}
        for year in range(1993, 2023):  
            joueurs_dict[loser_id].setdefault(str(year), {})

def calculate_percentage(value, total):
    return value / total if total != 0 else 0

#Met à jour les stats en prenant en compte les informations sur chaque match du joueur
def process_match(win,row, player_id, stats):
    stats['matchs'] += 1
    if win :
        stats['win'] += 1
        stats['pourc_serv_in'] += calculate_percentage(int(row['w_1stIn']), int(row['w_svpt']))
        stats['pourc_serv_win_pnt'] += calculate_percentage(int(row['w_1stWon']) + int(row['w_2ndWon']), int(row['w_svpt']))
        stats['pourc_return_win_pnt'] += calculate_percentage(int(row['l_svpt']) - int(row['l_1stWon']) - int(row['l_2ndWon']), int(row['l_svpt']))
        stats['pourc_serv_games_win'] += calculate_percentage(int(row['w_SvGms']) - (int(row['w_bpFaced']) - int(row['w_bpSaved'])), int(row['w_SvGms']))
        stats['pourc_break_games'] += 1 - calculate_percentage(int(row['l_SvGms']) - (int(row['l_bpFaced']) - int(row['l_bpSaved'])), int(row['l_SvGms']))
        stats['mean_ranking_oppo'] += int(row['loser_rank'])
        stats['pourc_break_point_made']+= calculate_percentage(int(row['l_bpFaced'])-int(row['l_bpSaved']),int(row['l_bpFaced']))
        stats['pourc_break_point_saved']+=calculate_percentage(int(row['w_bpSaved']),int(row['w_bpFaced']))
    else:
        stats['pourc_serv_in'] += calculate_percentage(int(row['l_1stIn']), int(row['l_svpt']))
        stats['pourc_serv_win_pnt'] += calculate_percentage(int(row['l_1stWon']) + int(row['l_2ndWon']), int(row['l_svpt']))
        stats['pourc_return_win_pnt'] += calculate_percentage(int(row['w_svpt']) - int(row['w_1stWon']) - int(row['w_2ndWon']), int(row['w_svpt']))
        stats['pourc_serv_games_win'] += calculate_percentage(int(row['l_SvGms']) - (int(row['l_bpFaced']) - int(row['l_bpSaved'])), int(row['l_SvGms']))
        stats['pourc_break_games'] += 1 - calculate_percentage(int(row['w_SvGms']) - (int(row['w_bpFaced']) - int(row['w_bpSaved'])), int(row['w_SvGms']))
        stats['mean_ranking_oppo'] += int(row['winner_rank'])
        stats['pourc_break_point_made']+= calculate_percentage(int(row['w_bpFaced'])-int(row['w_bpSaved']),int(row['w_bpFaced']))
        stats['pourc_break_point_saved']+=calculate_percentage(int(row['l_bpSaved']),int(row['l_bpFaced']))

def update_stats_for_player(player_id, stats, year):
    if stats['matchs'] > 0:
        for key in ['pourc_serv_in', 'pourc_serv_win_pnt', 'pourc_return_win_pnt', 'pourc_serv_games_win', 'pourc_break_games']:
            stats[key] = calculate_percentage(stats[key], stats['matchs'])

        for key in ['pourc_break_point_saved', 'pourc_break_point_made', 'mean_ranking_oppo']:
            stats[key] = calculate_percentage(stats[key], stats['matchs'])

        joueurs_dict[player_id][str(year)] = {
            'matchs': stats['matchs'],
            'win': stats['win'],
            'pourc_return_win_pnt': stats['pourc_return_win_pnt'],
            'pourc_break_games': stats['pourc_break_games'],
            'pourc_break_point_made': stats['pourc_break_point_made'],
            'pourc_break_point_saved': stats['pourc_break_point_saved'],
            'pourc_serv_games_win': stats['pourc_serv_games_win'],
            'pourc_serv_in': stats['pourc_serv_in'],
            'mean_ranking_oppo':stats['mean_ranking_oppo'],
            'pourc_serv_win_pnt':stats['pourc_serv_win_pnt']
        }



#on calcule les stats pour chaque joueur du dictionnaire
for year in range(1993,2022):
    for joueur in joueurs_dict.keys():
        
        stats = {
            'matchs': 0,
            'win': 0,
            'pourc_serv_win_pnt': 0,
            'pourc_return_win_pnt': 0,
            'pourc_serv_in': 0,
            'pourc_break_games': 0,
            'pourc_serv_games_win': 0,
            'pourc_break_point_saved': 0,
            'pourc_break_point_made': 0,
            'mean_ranking_oppo': 0
        }
        #pour éviter d'avoir un temps de calcul trop long, on se restreint à un dataset qui ne contient que les matchs de l'année voulue
        df_year = info_matches[(info_matches['tourney_date'] >= year * 10000) & (info_matches['tourney_date'] < (year + 1) * 10000)].copy()

        for i in range(len(df_year)):
            winner_id = df_year['winner_id'].iloc[i]
            loser_id = df_year['loser_id'].iloc[i]

            if winner_id == joueur :
                process_match(True,df_year.iloc[i], joueur, stats)

            elif loser_id == joueur :
                process_match(False,df_year.iloc[i], joueur, stats)


        update_stats_for_player(joueur, stats, year)

In [27]:
# Restructurer le dictionnaire pour obtenir une liste de dictionnaires
list_of_player_dicts = []
for player_id, player_data in joueurs_dict.items():
    player_data['id'] = player_id
    list_of_player_dicts.append(player_data)

# Utiliser pd.DataFrame pour convertir la liste de dictionnaires en DataFrame
df_info_joueurs = pd.DataFrame(list_of_player_dicts)

# Sauvegarder le DataFrame au format CSV
df_info_joueurs.to_csv('./Data/Data_utiles/info_joueurs.csv', index=False)




### On va maintenant ajouter les données que nous avons scrappé au fichier info_joueurs 

Principaux traitements effectués dans le code ci-dessous :

- $\textbf{Concaténation}$ des fichiers stats_return_1991_2022 et stats_under_pressure_1991_2022
- $\textbf{Ajout}$ du rang de chaque joueur (qui était avant cela contenu dans info_rank)
- $\textbf{Conversion}$ des pourcentages en décimaux (ce qui sera utile pour la partie ML)


### Problème rencontré : Conversion des élémenst de type dictionnaire en string

En effet, nous avons stockés les statistiques de chaque joueur sous forme de dictionnaires dans le dataset (un dictionnaire par année). Cependant le problème est que pandas convertit ces dictionnaires en chaine de caractères. Ce qui fait que lorsque nous souhaitons accéder à ceux-ci pour les modifier nous nous retrouvons avec un élément de type string et non un dictionnaire.

Pour résoudre ce problème, nous avons dû à chaque pour chaque joueur, récupérer le dictionnaire sous forme de chaine de caractère, le convertir en dictionnaire à nouveau avec le module $\textbf{eval}$, mettre à jours les éléments de dictionnaires puis l'ajouter à nouveau au dataset info_joueur.

In [4]:
df_info_retours=pd.read_csv('./Data/Data_utiles/stats_return_1991_2022.csv')
df_under_pressure=pd.read_csv('./Data/Data_utiles/stats_under_pressure_1991_2022.csv')
df_rang=pd.read_csv('./Data/Data_utiles/info_rank.csv')

df_info_joueurs=pd.read_csv('./Data/Data_utiles/info_joueurs.csv')



In [27]:
df_info_retours.head()

,Ranking,Name,,Return Rating,% 1 Serve Return Points Won,% 2nd Serve Return Points Won,% Return Games Won,% Break Points Converted,Year
0,1,Guillermo Perez-Roldan,NaN,183.3,37.5%,56.6%,38.6%,50.6%,1991
1,2,Magnus Gustafsson,NaN,177.2,37.9%,55.4%,36.1%,47.8%,1991
2,3,Thomas Muster,NaN,175.7,37.7%,54.4%,36.0%,47.6%,1991
3,4,Michael Chang,NaN,172.7,34.5%,54.8%,35.2%,48.2%,1991
4,5,Andrei Chesnokov,NaN,172.0,36.4%,52.7%,35.1%,47.8%,1991


In [28]:
df_under_pressure.head()

,Ranking,Name,,Under Pressure Rating,% Break Points Converted,% Break Point Saved,% Tie Breaks Won,% Deciding Sets Won,Year
0,1,Jaime Yzaga,NaN,249.0,44.0%,59.3%,85.7%,60.0%,1991
1,2,Sergi Bruguera,NaN,241.2,45.1%,59.8%,72.7%,63.6%,1991
2,3,Petr Korda,NaN,240.1,44.3%,56.8%,75.0%,64.0%,1991
3,4,Stefan Edberg,NaN,239.4,45.0%,64.5%,66.7%,63.2%,1991
4,5,Alberto Mancini,NaN,238.6,40.5%,60.1%,73.3%,64.7%,1991


In [3]:
df_info_joueurs.head()

,Unnamed: 0,name,hand,height,1993,1994,1995,1996,1997,1998,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,id
0,0,Richard Krajicek,R,196.0,"{'matchs': 53, 'win': 53, 'pourc_return_win_pn...","{'matchs': 44, 'win': 44, 'pourc_return_win_pn...","{'matchs': 62, 'win': 62, 'pourc_return_win_pn...","{'matchs': 71, 'win': 71, 'pourc_return_win_pn...","{'matchs': 65, 'win': 65, 'pourc_return_win_pn...","{'matchs': 60, 'win': 60, 'pourc_return_win_pn...",...,{},{},{},{},{},{},{},{},{},101990
1,1,Thomas Enqvist,R,190.0,"{'matchs': 38, 'win': 38, 'pourc_return_win_pn...","{'matchs': 41, 'win': 41, 'pourc_return_win_pn...","{'matchs': 83, 'win': 83, 'pourc_return_win_pn...","{'matchs': 78, 'win': 78, 'pourc_return_win_pn...","{'matchs': 54, 'win': 54, 'pourc_return_win_pn...","{'matchs': 50, 'win': 50, 'pourc_return_win_pn...",...,{},{},{},{},{},{},{},{},{},102358
2,2,Jonathan Stark,R,188.0,"{'matchs': 48, 'win': 48, 'pourc_return_win_pn...","{'matchs': 48, 'win': 48, 'pourc_return_win_pn...","{'matchs': 42, 'win': 42, 'pourc_return_win_pn...","{'matchs': 27, 'win': 27, 'pourc_return_win_pn...","{'matchs': 38, 'win': 38, 'pourc_return_win_pn...","{'matchs': 13, 'win': 13, 'pourc_return_win_pn...",...,{},{},{},{},{},{},{},{},{},101890
3,3,Todd Woodbridge,R,178.0,"{'matchs': 36, 'win': 36, 'pourc_return_win_pn...","{'matchs': 25, 'win': 25, 'pourc_return_win_pn...","{'matchs': 53, 'win': 53, 'pourc_return_win_pn...","{'matchs': 58, 'win': 58, 'pourc_return_win_pn...","{'matchs': 43, 'win': 43, 'pourc_return_win_pn...","{'matchs': 43, 'win': 43, 'pourc_return_win_pn...",...,{},{},{},{},{},{},{},{},{},101889
4,4,David Wheaton,R,193.0,"{'matchs': 52, 'win': 52, 'pourc_return_win_pn...","{'matchs': 40, 'win': 40, 'pourc_return_win_pn...","{'matchs': 51, 'win': 51, 'pourc_return_win_pn...","{'matchs': 19, 'win': 19, 'pourc_return_win_pn...","{'matchs': 11, 'win': 11, 'pourc_return_win_pn...","{'matchs': 10, 'win': 10, 'pourc_return_win_pn...",...,{},{},{},{},{},{},{},{},{},101609


In [5]:
# Liste de toutes les dernières semaines de chaque année
# Cela nous permet d'avoir le classement final
annees = [19931227, 19941226, 19951225, 19961230, 19971229, 19981228, 19991227, 20001225, 20011231, 20021230, 20031229, 20041227, 20051226, 20061225, 20071231, 20081229, 20091228, 20101227, 20111226, 20121231, 20131230, 20141229, 20151228, 20161226, 20171225, 20181231, 20191230, 20201228, 20211227, 20221226]

# Fonction pour convertir les pourcentages en décimaux dans un dictionnaire
def convert_percentages_to_decimals(data_dict):
    # On itère sur les clés et les valeurs du dictionnaire
    for key, value in data_dict.items():
        # On vérifie si la valeur est une chaîne contenant un pourcentage
        if isinstance(value, str) and '%' in value:
            # On supprime le signe pourcentage et on convertit en décimal
            value = float(value.rstrip('%')) / 100
        # On met à jour la valeur dans le dictionnaire
        data_dict[key] = value
    return data_dict

# Initialisation du compteur pour itérer sur la liste annees
compteur = 0

# On boucle sur les années
for year in range(1993, 2023):
    # On extrait les données de retours pour l'année en cours
    df_year_return = df_info_retours[df_info_retours['Year'] == year].copy()
    
    # On extrait les données de pression pour l'année en cours
    df_year_pressure = df_under_pressure[df_under_pressure['Year'] == year].copy()
    
    # On extrait le classement pour l'année en cours
    df_year_rang = df_rang[df_rang['ranking_date'] == annees[compteur]].copy()

    # On traite les données de retours
    for index, row in df_year_return.iterrows():
        # On normalise les noms en minuscules avant la comparaison
        normalized_name = row['Name'].lower()

        if normalized_name in df_info_joueurs['name'].str.lower().values:
            # On trouve l'indice du joueur
            indice = df_info_joueurs[df_info_joueurs['name'].str.lower() == normalized_name].index[0]

            # On trouve l'indice de la colonne dans le DataFrame
            indice_colonne = df_info_joueurs.columns.get_loc(str(year))

            # On convertit la chaîne en dictionnaire (si elle n'est pas déjà un dictionnaire)
            current_dict = eval(df_info_joueurs.at[indice, str(year)])

            # On met à jour les clés et les valeurs dans le dictionnaire
            current_dict.update(convert_percentages_to_decimals({
                'Return Rating': df_year_return['Return Rating'][index],
                ' % Serve Return Points Won': df_year_return['% 1 Serve Return Points Won'][index],
                ' % 2nd Serve Return Points Won': df_year_return['% 2nd Serve Return Points Won'][index],
                ' % Return Games Won': df_year_return['% Return Games Won'][index],
                ' % Break Points Converted': df_year_return['% Break Points Converted'][index],
            }))

            # On met à jour la cellule dans le DataFrame avec le nouveau dictionnaire
            df_info_joueurs.at[indice, str(year)] = str(current_dict)

    # On traite les données sous pression
    for index, row in df_year_pressure.iterrows():
        # On normalise les noms en minuscules avant la comparaison
        normalized_name = row['Name'].lower()

        if normalized_name in df_info_joueurs['name'].str.lower().values:
            # On trouve l'indice du joueur
            indice = df_info_joueurs[df_info_joueurs['name'].str.lower() == normalized_name].index[0]

            # On trouve l'indice de la colonne dans le DataFrame
            indice_colonne = df_info_joueurs.columns.get_loc(str(year))

            # On convertit la chaîne en dictionnaire (si elle n'est pas déjà un dictionnaire)
            current_dict = eval(df_info_joueurs.at[indice, str(year)])

            # On crée un nouveau dictionnaire avec les nouvelles clés et valeurs
            new_dict = convert_percentages_to_decimals({
                'Under Pressure Rating': df_year_pressure['Under Pressure Rating'][index],
                ' % Break Point Saved': df_year_pressure['% Break Point Saved'][index],
                ' % Break Points Converted Pressure': df_year_pressure['% Break Points Converted'][index],
                ' % Deciding Sets Won': df_year_pressure['% Deciding Sets Won'][index],
                ' % Tie Breaks Won': df_year_pressure['% Tie Breaks Won'][index],
            })

            # On met à jour la cellule dans le DataFrame avec le nouveau dictionnaire
            current_dict.update(new_dict)
            df_info_joueurs.at[indice, str(year)] = str(current_dict)

    # On traite le classement
    for index, row in df_year_rang.iterrows():
        normalized_name = row['Name'].lower()

        if normalized_name in df_info_joueurs['name'].str.lower().values:
            indice = df_info_joueurs[df_info_joueurs['name'].str.lower() == normalized_name].index[0]

            current_dict = eval(df_info_joueurs.at[indice, str(year)])
            if math.isnan(df_year_rang['points'][index]) :
                new_dict = {
                    'rang': df_year_rang['rank'][index],
                    'atp_points': 0
                }
            else :
                 new_dict = {
                    'rang': df_year_rang['rank'][index],
                    'atp_points':df_year_rang['points'][index]
                }

            # On met à jour le dictionnaire existant avec le nouveau dictionnaire
            current_dict.update(new_dict)

            # On convertit le dictionnaire en chaîne
            updated_str = str(current_dict)

            df_info_joueurs.at[indice, str(year)] = updated_str


    compteur =compteur +1


In [6]:
df_info_joueurs.to_csv('./Data/Data_utiles/info_joueurs.csv')

### On met tous les noms des joueurs en minuscules pour éviter les erreurs lors des recheches

In [7]:
df_info_joueurs=pd.read_csv('./Data/Data_utiles/info_joueurs.csv')

df_info_joueurs['name'] = df_info_joueurs['name'].str.lower()

# Sauvegarde du dataframe modifié dans un nouveau fichier CSV
df_info_joueurs.to_csv('./Data/Data_utiles/info_joueurs.csv', index=False)


# Création d'un dataset par année pour les stats des joueurs (utile pour la partie ML)


Ces datasets seront stockés dans Data/Data_utiles/Data_ML

In [8]:

# Lire le fichier CSV
df_info_joueurs = pd.read_csv('./Data/Data_utiles/info_joueurs.csv')


for year in range(1993,2022):
    # Sélectionner les colonnes pertinentes pour l'année spécifique
    df_temp = df_info_joueurs[['name', 'hand', 'height', str(year)]].copy()

    # Convertir la chaîne de caractères en dictionnaire à l'aide de literal_eval
    df_temp[str(year)] = df_temp[str(year)].apply(literal_eval)

    # Utiliser pd.json_normalize pour éclater les dictionnaires en colonnes
    df_from_dic = pd.json_normalize(df_temp[str(year)])
    
    
    df_temp.drop(str(year),axis=1,inplace=True)
    
    #On crée la colonne 'name' qui sera commune aux deux datasets et nous permettra donc de les merge ensemble
    df_from_dic['name']=df_temp['name']
    
    #On merge les deux datasets 
    df_temp = pd.merge(df_temp, df_from_dic, how='left',on='name')
    
    #On supprime les colonnes qui sont des doublons
    df_temp.dropna(subset=['matchs','rang'],inplace=True)

    
    df_temp.to_csv('./Data/Data_utiles/Data_ML/infos_joueurs_{}.csv'.format(year))
    

